In [1]:
# importing the libraries
import requests
import pandas as pd
from bs4 import BeautifulSoup
import time

In [2]:
# creating a list from years that I want to get the data
years = list(range(2023, 2018, -1))

In [3]:
# a empty list to receive every team data
all_matches = []

In [4]:
# URL of Fbref 
standings_url = "https://fbref.com/en/comps/24/Serie-A-Stats"

In [7]:
# Looping to get the data from all clubs from 2019 to 2023

for year in years:
    data = requests.get(standings_url) # requesting the URL to web scraping.
    soup = BeautifulSoup(data.text)
    standings_table = soup.select('table.stats_table')[0] # select the table in Fbref

    links = [l.get("href") for l in standings_table.find_all('a')] # getting the links for every team in Serie A
    links = [l for l in links if '/squads/' in l]
    team_urls = [f"https://fbref.com{l}" for l in links]
    
    previous_season = soup.select("a.prev")[0].get("href") # searching for previous seasons to get the data.
    standings_url = f"https://fbref.com{previous_season}"
    
    for team_url in team_urls:
        team_name = team_url.split("/")[-1].replace("-Stats", "").replace("-", " ") # extracting the team name
        data = requests.get(team_url)
        matches = pd.read_html(data.text, match="Scores & Fixtures")[0]
        soup = BeautifulSoup(data.text)
        links = [l.get("href") for l in soup.find_all('a')]
        links = [l for l in links if l and 'all_comps/shooting/' in l]
        data = requests.get(f"https://fbref.com{links[0]}")
        shooting = pd.read_html(data.text, match="Shooting")[0]
        shooting.columns = shooting.columns.droplevel()
        try:
            team_data = matches.merge(shooting[["Date", "Sh", "SoT","SoT%", "G/Sh", "G/SoT","PK", "PKatt"]], on="Date")
        except ValueError:
            continue
                
        team_data["Season"] = year
        team_data["Team"] = team_name
        all_matches.append(team_data) # append all team data to a list
        time.sleep(1)
   

In [8]:
# concating all tables, and saving in csv:
match_df = pd.concat(all_matches)
match_df.columns = [c.lower() for c in match_df.columns]
match_df.to_csv("Dados_Brasileirão_2019_2023.csv", encoding='utf-8-sig', index=False)